In [1]:
import pandas as pd
import numpy as np

# Merge and Clean the Water, Soil, Radiation and Weather Data 

In [37]:
# read in files
weather = pd.read_csv("../../Data/raw_data/mythenquai_agg.csv")
soil = pd.read_csv("../../Data/raw_data/saugspannung_agg.csv")
water = pd.read_csv("../../Data/raw_data/waterdata_agg.csv")

In [34]:
# Merge the three datasets
df_merged = pd.merge(pd.merge(soil, weather, how="left", on="datetime"), water, how="left", on="datetime")
df_merged

,datetime,Wädenswil (20cm),Wädenswil (40cm),Wädenswil (70cm),air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,...,precipitation,global_radiation,change_rate,Pegel Zurichsee,Pegel Obersee,Abfluss Linth,Pegel Linth,Wasser_temp Linth,Abfluss Limmat,Pegel Limmat
0,2012-01-01 01:00:00+01:00,0.50,0.60,1.000,7.516667,5.9,4.783333,2.516667,2.516667,194.333333,...,0.1,0.0,0.000008,405.877,405.850,29.328,418.226,6.16,134.098,400.669
1,2012-01-01 02:00:00+01:00,0.60,0.70,1.000,7.333333,5.9,5.266667,3.266667,3.266667,179.666667,...,0.0,1.0,0.000016,405.883,405.855,29.543,418.231,6.17,143.515,400.732
2,2012-01-01 03:00:00+01:00,0.70,0.80,1.200,9.116667,5.9,6.316667,2.650000,2.650000,216.000000,...,0.0,9.0,0.000004,405.887,405.857,29.741,418.236,6.17,158.646,400.830
3,2012-01-01 04:00:00+01:00,0.80,0.80,1.200,9.716667,5.9,6.900000,2.900000,2.900000,216.000000,...,0.0,8.0,0.000016,405.891,405.858,30.058,418.243,6.18,165.654,400.874
4,2012-01-01 05:00:00+01:00,1.00,0.90,1.300,9.966667,5.9,8.050000,3.366667,3.366667,218.166667,...,0.0,2.0,0.000004,405.894,405.858,30.277,418.249,6.18,163.086,400.858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73844,2021-12-30 21:00:00+01:00,2.37,2.55,5.505,9.900000,NaN,1.883333,1.416667,1.333333,168.500000,...,0.0,0.0,0.000000,405.953,405.926,49.147,418.675,6.96,149.153,400.736
73845,2021-12-30 22:00:00+01:00,2.40,2.58,5.525,9.216667,NaN,2.500000,2.016667,2.000000,168.500000,...,0.0,0.0,0.000000,405.954,405.928,49.756,418.687,6.93,149.750,400.740
73846,2021-12-30 23:00:00+01:00,2.44,2.61,5.535,8.583333,NaN,2.366667,1.850000,2.000000,161.500000,...,0.0,0.0,0.000000,405.955,405.927,50.410,418.700,6.91,149.250,400.737
73847,2021-12-31 00:00:00+01:00,2.50,2.64,5.560,8.750000,NaN,2.250000,1.750000,1.666667,152.666667,...,0.0,0.0,0.000000,405.956,405.924,51.048,418.713,6.89,147.866,400.728


In [35]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73849 entries, 0 to 73848
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   datetime                 73849 non-null  object 
 1   Wädenswil (20cm)         73622 non-null  float64
 2   Wädenswil (40cm)         73737 non-null  float64
 3   Wädenswil (70cm)         73454 non-null  float64
 4   air_temperature          73398 non-null  float64
 5   water_temperature        60619 non-null  float64
 6   wind_gust_max_10min      73398 non-null  float64
 7   wind_speed_avg_10min     73398 non-null  float64
 8   wind_force_avg_10min     73398 non-null  float64
 9   wind_direction           73398 non-null  float64
 10  windchill                73398 non-null  float64
 11  barometric_pressure_qfe  73394 non-null  float64
 12  dew_point                73398 non-null  float64
 13  humidity                 73398 non-null  float64
 14  water_level           

In [40]:
# select all observation until the end of 2019, from 2020 onwards there are no water level and water temperature data anymore furthermore we only have solar radiation Data
# until the end of 2019 (we will read in that dataset further down) 
df = df_merged.loc[df_merged["datetime"]<"2019-12-31 23:00:00+01:00"]
df.drop(columns=["wind_direction", "wind_gust_max_10min", "wind_force_avg_10min", "windchill", "Pegel Zurichsee", "Pegel Obersee", "Pegel Zurichsee", "Pegel Linth", "Pegel Limmat", "Wasser_temp Linth"], inplace=True)

# rename the columns
df.rename(columns={"Wädenswil (20cm)": "suction_tension (20cm)", "Wädenswil (40cm)": "suction_tension (40cm)", "Wädenswil (70cm)": "suction_tension (70cm)",
 "wind_speed_avg_10min": "wind_speed (m/s)", "precipitation": "precipitation (mm)","Abfluss Linth": "inflow_linth (m3/s)",
                   "Abfluss Limmat": "outflow_limmat (m3/s)"}, inplace=True)


C:\Users\lucal\AppData\Local\Temp\ipykernel_8812\2382665581.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["wind_direction", "wind_gust_max_10min", "wind_force_avg_10min", "windchill", "Pegel Zurichsee", "Pegel Obersee", "Pegel Zurichsee", "Pegel Linth", "Pegel Limmat", "Wasser_temp Linth"], inplace=True)
C:\Users\lucal\AppData\Local\Temp\ipykernel_8812\2382665581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Wädenswil (20cm)": "suction_tension (20cm)", "Wädenswil (40cm)": "suction_tension (40cm)", "Wädenswil (70cm)": "suction_tension (70cm)",


In [46]:

# impute the missing values of the dataset using spline interpolate imputation. We are taking the mean of the two values before and after the 
# observation with the missing value to get the best possible approximation. The interpolate imputation is the best method to impute 
# and we chose the spline method because that way we can smooth the estimations since we don´t have linear data
# source: https://www.section.io/engineering-education/missing-values-in-time-series/ 

df[["suction_tension (20cm)", "suction_tension (40cm)", "suction_tension (70cm)", "air_temperature", "water_temperature", "wind_speed (m/s)", 
"barometric_pressure_qfe", "dew_point", "humidity","water_level", "inflow_linth (m3/s)","outflow_limmat (m3/s)"]]= df[["suction_tension (20cm)", 
"suction_tension (40cm)", "suction_tension (70cm)", "air_temperature", "water_temperature", "wind_speed (m/s)", "barometric_pressure_qfe", 
"dew_point", "humidity","water_level", "inflow_linth (m3/s)","outflow_limmat (m3/s)"]].interpolate(option='spline')


C:\Users\lucal\AppData\Local\Temp\ipykernel_8812\736777462.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["suction_tension (20cm)", "suction_tension (40cm)", "suction_tension (70cm)", "air_temperature", "water_temperature", "wind_speed (m/s)",


In [47]:
# update the change rate variable after imputing the water level variable
df["change_rate"] = df["water_level"].pct_change()

C:\Users\lucal\AppData\Local\Temp\ipykernel_8812\132330667.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["change_rate"] = df["water_level"].pct_change()


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60754 entries, 0 to 60753
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   datetime                 60754 non-null  object 
 1   suction_tension (20cm)   60754 non-null  float64
 2   suction_tension (40cm)   60754 non-null  float64
 3   suction_tension (70cm)   60754 non-null  float64
 4   air_temperature          60754 non-null  float64
 5   water_temperature        60754 non-null  float64
 6   wind_speed (m/s)         60754 non-null  float64
 7   barometric_pressure_qfe  60754 non-null  float64
 8   dew_point                60754 non-null  float64
 9   humidity                 60754 non-null  float64
 10  water_level              60754 non-null  float64
 11  precipitation (mm)       60754 non-null  float64
 12  global_radiation         60754 non-null  float64
 13  change_rate              60753 non-null  float64
 14  inflow_linth (m3/s)   

# Clean radiation dataset and merge with the other dataset

In [53]:
# read in dataset
RA = pd.read_csv("../../Data/raw_data/extrater_radiation_agg.csv")

In [54]:
# rename column and merge the two dataframes
RA.rename(columns={"Irradiance (W/m2)": "extraterrestial_radiation"}, inplace=True)
df_RA = pd.merge(df, RA, how="inner", on="datetime")

In [55]:
# check if there are any missing values after the merge
df_RA["extraterrestial_radiation"].isna().unique()

array([False])

In [56]:
df_RA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60754 entries, 0 to 60753
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   datetime                   60754 non-null  object 
 1   suction_tension (20cm)     60754 non-null  float64
 2   suction_tension (40cm)     60754 non-null  float64
 3   suction_tension (70cm)     60754 non-null  float64
 4   air_temperature            60754 non-null  float64
 5   water_temperature          60754 non-null  float64
 6   wind_speed (m/s)           60754 non-null  float64
 7   barometric_pressure_qfe    60754 non-null  float64
 8   dew_point                  60754 non-null  float64
 9   humidity                   60754 non-null  float64
 10  water_level                60754 non-null  float64
 11  precipitation (mm)         60754 non-null  float64
 12  global_radiation           60754 non-null  float64
 13  change_rate                60753 non-null  flo

In [57]:
# this is the simplified version of the penman equation to compute evaporation based on global radiation (RS), air teperature (T), extraterrestial radiation (RA), 
# humidity (RH), and wind speed (U)
# we derived this formula from the following source: Valiantzas, J.D. (2006). Simplified versions for the Penman evaporation equation using routine weather data, 
# Journal of Hydrology, 331(3–4), 690-702. https://doi.org/10.1016/j.jhydrol.2006.06.012.
 
def evaporation(RS,T,RA,RH,U, alpha=0.08, au=0.5):
  Epen = (0.051*(1-alpha)*RS*np.sqrt(T+9.5))-(2.4*(RS/RA)**2)+(0.052*(T+20)*(1-(RH/100))*(au-0.38+0.54*U))
  return Epen

In [58]:
# create new variable evaporation by applying the evaporation function
df_RA["evaporation"] = evaporation(df_RA["global_radiation"], df_RA["air_temperature"], df_RA["extraterrestial_radiation"], df_RA["humidity"], df_RA["wind_speed (m/s)"])

c:\Users\lucal\miniconda3\envs\dsf\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [59]:
# set datetime as index
df_RA.set_index("datetime", inplace=True)

In [61]:
# For the missing values of the evaporation we are using the KNNImputer to fill in the missing values.
# we use the KNN imputer because it fills in the value based of the mean values of observations with variable values most similar to the observation
# with the missing value. 
# The NaNs occure for observations with a very low (ca. below -9 degrees Celsius) temperature because then the argument in the square root
# turn negative. Because of this we often have missing values for many consecutive observations in a row and because we have all other values
# for of one observation the KNN imputer makes more sense than interpolation 

# We are standardizing the values before we apply the KNN imputer and destandardize them afterwards again. That way we circumvent biases for the
# imputed values
# source: Htoon, K. S. (2020).  A Guide To KNN Imputation. retrieved from: 
# https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e

# import KNN imputer
from sklearn.impute import KNNImputer
# import MinMaxScaler 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# standardize data
data_scaled = pd.DataFrame(scaler.fit_transform(df_RA), columns = df_RA.columns)

imputer = KNNImputer()
# impute missing values with KNN imputer and convert them to floats
data_imputed = pd.DataFrame(imputer.fit_transform(data_scaled), columns=df_RA.columns, index = df_RA.index).astype(float)
# de standardize the imputed dataframe to get original unit of measurement
data_imputed = pd.DataFrame(scaler.inverse_transform(data_imputed), columns = df_RA.columns, index = df_RA.index)


In [64]:
data_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60754 entries, 2012-01-01 01:00:00+01:00 to 2019-11-27 05:00:00+01:00
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   suction_tension (20cm)     60754 non-null  float64
 1   suction_tension (40cm)     60754 non-null  float64
 2   suction_tension (70cm)     60754 non-null  float64
 3   air_temperature            60754 non-null  float64
 4   water_temperature          60754 non-null  float64
 5   wind_speed (m/s)           60754 non-null  float64
 6   barometric_pressure_qfe    60754 non-null  float64
 7   dew_point                  60754 non-null  float64
 8   humidity                   60754 non-null  float64
 9   water_level                60754 non-null  float64
 10  precipitation (mm)         60754 non-null  float64
 11  global_radiation           60754 non-null  float64
 12  change_rate                60754 non-null  float64
 13  inflow_

In [69]:
# safe dataframe with all variables as csv
data_imputed.to_csv("../../Data/clean_data/data_hourly_all_variables.csv", index=True)

In [65]:
# drop variables that are included in the evaporation
df_reduced = data_imputed.drop(columns=["air_temperature", "wind_speed (m/s)", "global_radiation", "humidity", "extraterrestial_radiation"])


In [70]:
# safe dataframe with reduced variabels as csv
df_reduced.to_csv("../../Data/clean_data/data_hourly_reduced_variables.csv", index=True)